# Case Study 2 - Sheep Femur Trabecular Bone

CRBS VectoRose Bootcamp

Benjamin Z. Rudski <benjamin.rudski@mail.mcgill.ca>

August 6, 2025

In the previous case study notebook, we saw how to use VectoRose for analysing pure orientations. We constructed spherical orientation histograms and filtered based on orientation.

Now, we're going to kick things up a notch.

What if your data aren't pure directions or orientations? What if your vectors have some sort of **magnitude** associated with them? How can we visualise these quantities?

Stay tuned to find out!

In this notebook, we'll explore these additional functionalities of VectoRose through the analysis of trabecular bone anisotropy in a sheep femur.

## Outline

Before we begin, here's the outline of the material we'll cover in this notebook.

1. Sample Data
2. Review: Getting Started
3. Preparing the Sphere
4. Constructing Magnitude Histograms
5. Constructing Bivariate Nested Spherical Histograms
6. Animating Bivariate Histograms
7. Constructing Conditional Histograms
8. Note on Computing Statistics

After following this notebook (in addition to the previous one), you will be familiar with almost all the basic functionality of VectoRose. You will then be able to apply VectoRose to analyse your data, regardless of the sample under study or the data acquisition method used.

## Sample Data

Bones are not fully-filled structures. Inside bone, there is a network of dense, interconnected struts, known as *trabeculae*. 

In this example, we'll focus on the trabecular bone in the femur of a mature sheep. Trabecular bone isn't found throughout the entire bone. Instead, in long bones, like the femur, it's found at the two ends, or *epiphyses*.

Through the worked example, we'll analyse the trabecular bone in the *proximal* epiphysis, the end that is closest to the hip. Later, you'll be able to analyse the trabecular bone in the *distal* epiphysis, which is part of the knee.

The entire sheep femur is a very, very big dataset. Instead of looking at the entire volume here, we'll look at two different meshes for a start. We'll look at the exterior shell of the femur, just to get a feel for the bone, and then, we'll look at some simplified cubes of trabecular bone to understand this structure a bit better.

In [ ]:
from IPython.display import Video

Video("../assets/CaseStudy2/SheepFemur.mp4", width=720)

> 📝 **Note**\
> We will analyse larger volumes of the bone to understand the trabecular architecture. This simplification was just for the purposes of illustration.

### Trabecular Anisotropy Overview

One last thing that we need to go over is... well... the actual vector collection.

The bone is just an image! Where do the vectors come in? Well, we'll look at **trabecular anisotropy**, which is a measurement of the local co-alignment of the trabecular struts.

This quantity provides us **two** pieces of information:

1. the dominant local orientation of trabeculae in a certain volume
2. the degree of co-alignment of the trabeculae

Trabecular anisotropy can be thought of as a **vector field**; at each spatial location, we have a vector pointing in the dominant local orientation with a magnitude corresponding to the degree of co-alignment.

So, in this case, we don't just want to look at the local orientations! We also want to consider the associated magnitudes.

Let's see how to take care of this type of data using VectoRose.

## Review: Getting Started

As we did before, we need to start by loading our vector data into Python using VectoRose, and we need to perform some data cleaning.

Let's do all the steps in one block this time.

In [ ]:
vectors_path = "../assets/CaseStudy2/MatureSheepFemur_Proximal_S750_R1500_SMOOTH1_EIG_MASKED.npy"

# Your code here for vector loading and preprocessing


## Preparing the Sphere

Now, we need to create a sphere to build our histograms. Like last time, we'll use a [`FineTregenzaSphere`](https://vectorose.readthedocs.io/en/latest/autoapi/vectorose/tregenza_sphere/index.html#vectorose.tregenza_sphere.FineTregenzaSphere).

But we'll do something different when we create it!

Since we want to look at the magnitude distribution also, we're going to create more than one shell. Let's consider 10 magnitude bins. When creating the `FineTregenzaSphere`, we need to set `number_of_shells=10`.

By default, the highest and lowest bin edges will be determined based on the magnitudes present in the data. We can change these bounds using the `magnitude_range` parameter. In the case of trabecular anisotropy, values can only be in the interval `[0, 1]`, where an anisotropy of 0 represents structures with no co-alignment and 1 represents structures with perfect co-alignment. We'll use these as the bounds for our `magnitude_range`, as well.

In [ ]:
# Your code here to create the Tregenza sphere


Now, we have a sphere that we can use to bin by both magnitude and orientation!

Like before, the next step is to assign the vectors to histogram bins. Since our sphere has multiple magnitude shells, the [`assign_histogram_bins`](https://vectorose.readthedocs.io/en/latest/autoapi/vectorose/sphere_base/index.html#vectorose.sphere_base.SphereBase.assign_histogram_bins) method takes care of both the magnitude and the orientation. This method produces two outputs:

1. the `DataFrame` with the bin assignments
2. the edges of the magnitude bins

We haven't seen the second returned value yet, but this is important. It tells us where each magnitude bin starts and ends. These will be very important when we want to plot our magnitude histograms.

So, let's now assign our vectors to bins!

In [ ]:
# Your code here to assign vectors to their bins


## Constructing Magnitude Histograms

At first, the table above looks quite similar to what we had last time... but notice that the **`shell`** column now doesn't contain all zeros. We have a variety of different values in there.

The `shell` column contains the bin indices for the magnitude histogram! We're going to use these bin assignments to construct a magnitude histogram.

Using VectoRose, we can easily collect all these bin assignments into a histogram using the method [`construct_marginal_magnitude_histogram`](https://vectorose.readthedocs.io/en/latest/autoapi/vectorose/sphere_base/index.html#vectorose.sphere_base.SphereBase.construct_marginal_magnitude_histogram) on our `FineTregenzaSphere`.

Let's see what this produces.

In [ ]:
# Your code here to compute the magnitude histogram


We can see that we now have the fraction of vectors that are associated with each magnitude histogram bin.

> 💡 **Tip**\
> Interested in counts instead of frequencies? Set `return_fraction=False` when calling this method, and you'll get a histogram based on counts instead.

This is great... but we want a nice bar plot, not just a list of numbers.

Thankfully, using [`vr.plotting.produce_1d_scalar_histogram`](https://vectorose.readthedocs.io/en/latest/autoapi/vectorose/plotting/index.html#vectorose.plotting.produce_1d_scalar_histogram), we can get our histogram plot.

This function requires the histogram data, as well as those bin boundaries we computed before.

Let's now visualise our magnitude histogram!

In [ ]:
# Your code here to visualise the magnitude histogram


This plot is generated with [Matplotlib](https://matplotlib.org/), and so, we can apply all the usual Matplotlib customisations to it, like adding titles and axis labels.

Another helpful thing we can do is set the *y*-axis to use a logarithmic scale by passing `log=True` to the function call.

Let's see what happens for these data.

In [ ]:
# Your code here to construct a log histogram


Now we successfully have a log-scale plot!

## Constructing Bivariate Histograms

So, by now we've seen how to construct histograms of just orientation or just magnitude... but what if we want to see which orientations are more common for high magnitudes, or for low magnitudes, what can we do?

Well, we can construct **bivariate histograms** of magnitude and orientation. These histograms show a frequency value for each **combination** of magnitude and orientation.

To visualise these combinations, we rely on **nested spherical histograms**. Each level in the spherical histogram corresponds to a different range of magnitude values.

### Computing Bivariate Histograms

As always, before we can visualise a histogram, we need to compute the bin frequencies. For the bivariate histograms, we do this using the [`construct_histogram`](https://vectorose.readthedocs.io/en/latest/autoapi/vectorose/sphere_base/index.html#vectorose.sphere_base.SphereBase.construct_histogram) method on our `FineTregenzaSphere`.

As usual, if we want to work with counts instead of frequencies, we can pass `return_fraction=False`.

In [ ]:
# Your code here to compute the bivariate histogram


We now have something that looks similar to the orientation histogram, but with a twist. We now have an extra layer of indexing. We see the frequency associated with each ring and bin in each shell.

### Visualising the Bivariate Histogram

Now, to visualise the histogram, we'll once again use the [`SpherePlotter`](https://vectorose.readthedocs.io/en/latest/autoapi/vectorose/plotting/index.html#vectorose.plotting.SpherePlotter). But first, we need to provide histogram meshes.

To construct histogram meshes for the bivariate histogram, we must pass the histogram data to the method [`create_histogram_meshes`](https://vectorose.readthedocs.io/en/latest/autoapi/vectorose/sphere_base/index.html#vectorose.sphere_base.SphereBase.create_histogram_meshes). 

This method produces one spherical mesh for each histogram shell.

There is one other required parameter: the magnitude bins. These bin bounds are used to set the size of the produced spheres:

* If the bin edges found above are provided, then they are used to determine the radius of each sphere. The sphere radii correspond to the **upper bound** of the bins.
* If `None` is provided, then the sphere all have the common radius of one.

Let's start by providing the bin edges.

In [ ]:
# Your code here to create the histogram meshes


We can now see that we have a number of meshes that have been produced. If we look at the bounds for each in their descriptions, we see that they get progressively larger.

Now, let's use the `SpherePlotter` to visualise these histogram shells.

> ⚠️ **Caution**\
> As always, don't forget to call `SpherePlotter.produce_plot` before trying to `show` the plot.

In [ ]:
# Your code here to create the sphere plotter


While this plot initially looks pretty similar to the orientation histogram, you may notice an important difference. At the top, instead of having a single slider, controlling the opacity of the histogram, we now have three sliders.

Try playing with each one. You'll notice that:

* The left slider controls which shell is currently active.
* The central slider controls the opacity of this active shell.
* The right slider controls the opacity of all other inactive shells.

At the extreme case, if the central slider is set to `1` and the right slider is set to `0`, we only have one shell visible at a time. The visible shell is determined by the left slider.

So, we can get a pretty clear view of the vector distribution by flipping through these different shells.

Wouldn't it be nice if there were an easier way to do this?

## Animating Bivariate Histograms

In addition to the rotation animations we covered in the previous notebook, we can also create bivariate histogram shell animations. In these animations, each frame contains a different spherical shell.

To create a shell animation, we call the method [`produce_shells_video`](https://vectorose.readthedocs.io/en/latest/autoapi/vectorose/plotting/index.html#vectorose.plotting.SpherePlotter.produce_shells_video) on the `SpherePlotter`.

In [ ]:
# Your code here to produce a shell animation video


In [ ]:
Video("shells_animation.mp4", width=720, html_attributes="loop autoplay controls")

> 💡 **Tip**\
> Unlike the rotation videos, the shell videos only capture a single perspective of the sphere. You may want to rotate the spherical histogram and capture a shell video of the back to ensure that you have viewed the entire surface.

## Constructing Conditional Histograms

The bivariate histograms are very helpful for showing us the frequency of all orientations at all magnitudes.

But what if we only want to look at how higher-magnitude vectors are oriented?

There are a couple of different approaches we can use:

1. Filtering using tools in [pandas](https://pandas.pydata.org/).
2. Computing conditional histograms with VectoRose.

Right now, we'll focus on the second approach.

Recall that a **conditional distribution** looks at the frequency of one variable given certain values for the other variable.

### Conditioning Orientation on Magnitude

Let's say we want to look at the orientation distribution of vectors with a magnitude in a certain range. In this case, we look at the frequency of orientations **only among vectors with the specified magnitudes**.

We're *conditioning* orientation on magnitude.

To do this, we can use the method [`construct_conditional_orientation_histogram`](https://vectorose.readthedocs.io/en/latest/autoapi/vectorose/sphere_base/index.html#vectorose.sphere_base.SphereBase.construct_conditional_orientation_histogram) of the `FineTregenzaSphere`.

> ⚠️ **Attention**\
> Unlike the other histogram methods, we can only get the frequencies as fractions in conditional histograms. We can't get the results in terms of counts.

In [ ]:
# Your code here to compute the conditional orientation histogram

conditional_orientation_histogram = my_sphere.construct_conditional_orientation_histogram(
    labelled_vectors
)

conditional_orientation_histogram.to_frame()

At first glance, this looks pretty similar to the bivariate histogram. But the important difference is that the frequencies are normalised by shell.

Within each shell, the frequencies sum to 1 (unless there are no vectors in that shell).

Let's look, for example, at shell 6 (index 5). We'll extract this shell using pandas indexing.

In [ ]:
# Your code here to select shell index 5


We now have an orientation histogram for a single shell! We can now use this to generate a spherical orientation histogram like we did in the previous notebook.

In [ ]:
# Your code here to generate a spherical orientation histogram


Notice that now we can see the features in this sphere more clearly. We can now see orientation patterns that are present in specific shells that may otherwise be invisible due to relatively low frequencies.

What if we want to do the opposite, now?

We can also look at the distribution of magnitude values assuming vectors have a certain orientation. In this case, we'd look at the frequency of magnitudes **only among vectors with the specified orientations**.

We are *conditioning* magnitude on orientation.

We can compute the conditional orientation distribution using the method [`construct_conditional_magnitude_histogram`](https://vectorose.readthedocs.io/en/latest/autoapi/vectorose/sphere_base/index.html#vectorose.sphere_base.SphereBase.construct_conditional_magnitude_histogram) on the `FineTregenzaSphere`.

But, this approach may be overly fine-grained, as it considers each face on its own... Instead, let's do some manual filtering.

First, we need to have some orientations to use for filtering.

To do this, let's once again turn to interactive cell picking! We'll quickly create a marginal orientation histogram and pick cells on it.

In [ ]:
# Your code here for creating and visualising the marginal orientation histogram


We can see that there are a lot of vectors near the pole. Let's pick some of those bright patches.

> ⚠️ **Attention**\
> Remember to pick the cells by **right-clicking**.

Now that we have some cells selected, we can extract the vectors contained within these orientations.

In [ ]:
# Your code here to get the vectors in the selected cells


Now, we can just construct a magnitude histogram like we did above using the `construct_marginal_magnitude_histogram` method, only using these `selected_vectors` instead of all the `labelled_vectors`.

In [ ]:
# Your code here to build the magnitude histogram

conditional_magnitude_hist = my_sphere.construct_marginal_magnitude_histogram(
    selected_vectors
)

ax = vr.plotting.produce_1d_scalar_histogram(
    conditional_magnitude_hist, magnitude_bin_edges
)

We can see that our magnitude distribution among the directions we've selected is different from that of all the data! These can reveal interesting insights into the relationship between magnitude and orientation, revealing potentially interesting biological findings!

## Note on Computing Statistics

Up until now, all our analysis have been visual. We've been looking at the magnitude distribution, the bivariate distribution, and the conditional distributions...

But what if we want to get some numerical insights?

Well, the magnitude data are scalars, so we can just analyse both the marginal and conditional magnitude distributions using standard statistics (if the context allows for it).

Similarly, we can just analyse conditional orientation distributions with directional statistics approaches. We saw some of these approaches that are implemented in VectoRose in the previous notebook.

What about the bivariate distribution?

VectoRose can compute the correlation between the orientation and magnitude data. If the two variables are statistically independent, they will not be correlated.

We can compute the correlation using the function [`vr.stats.compute_magnitude_orientation_correlation`](https://vectorose.readthedocs.io/en/latest/autoapi/vectorose/stats/index.html#vectorose.stats.compute_magnitude_orientation_correlation).

This function returns both the correlation coefficient and the result of the hypothesis test (containing the *p*-value).

As before, we need an array containing our vectors. Unlike previously, these vectors **should not be unit vectors** (otherwise, there won't be any correlation).

We could do this once again with the method [`convert_vectors_to_cartesian_array`](https://vectorose.readthedocs.io/en/latest/autoapi/vectorose/sphere_base/index.html#vectorose.sphere_base.SphereBase.convert_vectors_to_cartesian_array) of the `FineTregenzaSphere`, but we don't have to since we have our **unduplicated** vectors already stored in an array.

In [ ]:
# Your code here to compute correlation


We can see a pretty strong correlation between the magnitude and the orientation, allowing us to reject the null hypothesis that magnitude and orientation are independent.

## Conclusion

We've now reached the end of this notebook on using VectoRose to analyse trabecular anisotropy. Here's a recap of what we saw:

* Using VectoRose, we can examine vectors that have **different magnitudes**.
* We can construct **magnitude histograms** showing the distribution of vector lengths.
* We can construct **bivariate histograms** showing the interplay of magnitude and orientation. We can produce **animations** showing the different histogram shells.
* We can construct **conditional orientation and magnitude histograms** which show the distribution of one variable while filtering based on the other one.
* We can **compute statistics** on the magnitudes and filtered data and compute the **correlation** between magnitude and orientation.

At this point, we've covered the fundamentals of using VectoRose. You should be able to apply VectoRose to studying different collections of vectors, regardless of whether they are of unit or non-unit length.

## Extra Practice

In this notebook, we've been exploring the proximal epiphysis of the mature sheep femur. What about the other end of the bone? I've provided the anisotropy field for the distal epiphysis in the file [`../assets/CaseStudy2/MatureSheepFemur_Distal_S750_R1500_SMOOTH1_EIG_MASKED.npy`](../assets/CaseStudy2/MatureSheepFemur_Distal_S750_R1500_SMOOTH1_EIG_MASKED.npy). Look at it in terms of the magnitude and bivariate distributions, and then explore any interesting features using conditional plots. Does the distal end look different from the proximal end of this bone?

In [ ]:
# Your analysis code here


## Additional Resources

Though this is the end of the Bootcamp material, there are more resources out there for learning about VectoRose. Make sure to check out the online documentation at <https://vectorose.readthedocs.io/en/latest/index.html>. This resource contains:

* **API Reference** - explanation of all classes and functions included in VectoRose.
* **User's Guide** - narrative explanations and code examples for using VectoRose.

Make sure to check out the documentation! I wrote it to make it easier for you to learn how to use VectoRose and to explore what you can do with it.

If you have any questions, feel free to reach out to me. If you find any software bugs, please open an issue on [GitHub](https://github.com/bzrudski/vectorose).

Thank you very much for your participation!